In [ ]:
import pretty_midi_fluidsynth as pmf
import librosa
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio

from PIL import Image
import soundfile as sf

In [ ]:
sr = 44100

# hop_len = 512
# freq_w = 3
# octave = 8

fs = 100

# gen data save

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm

- Class 1 (Acoustic Grand Piano(0) ~ Dulcimer(15))
- Class 2 (Drawbar Organ(16) ~ Synth Choir(54))
- Class 3 (Orchestra Hit(55) ~ Shakuhachi(77))
- Class 4 (Whistle(78) ~ Gunshot(127))

In [ ]:
pmf.instrument_name_to_program('Gunshot')

In [ ]:
instr_range = {
    "class1":(0,16),
    "class2":(16,55),
    "class3":(55,78),
    "class4":(78,128),
}

In [ ]:
def mk_dir(key):
    base_path = Path('./gen_data','key')
    base_path.mkdir(parents=True, exist_ok=True)

    x_path = Path(base_path,'wav')
    x_path.mkdir(parents=True, exist_ok=True)

    y_path = Path(base_path,'midi')
    y_path.mkdir(parents=True, exist_ok=True)
    return base_path,x_path,y_path

In [ ]:
sf_list = os.listdir("./sf")
print(sf_list)

In [ ]:
for sf_file in sf_list:
    print(sf_file.split('.')[0])

In [ ]:
for key, value in instr_range.items():
    base_path, x_path, y_path = mk_dir(key)
    for sf_file in sf_list[:]:
        for idx in tqdm(range(10)):
            instr_list = []

            n = np.random.randint(1,10)
            for _ in range(n):
                i = np.random.randint(*value)
                instr = pmf.Instrument(program=i)
                instr.name = pmf.program_to_instrument_name(i)

                note_n = np.random.randint(2,10)
                velocity = np.random.randint(50,125)

                for _ in range(note_n):
                    pitch = np.random.randint(24,120)
                    start_time = np.random.randint(1,10)
                    end_time = start_time + np.random.randint(1,4)
                    note = pmf.Note(velocity=velocity,
                                   pitch=pitch,
                                   start=start_time,
                                   end=end_time,)
                    instr.notes.append(note)
                instr_list.append(instr)

            pm = pmf.PrettyMIDI()
            for instr in instr_list:
                pm.instruments.append(instr)

            sf_path = os.path.join('./sf/',sf_file)
            x = pm.fluidsynth(sr,sf2_path=sf_path)

            filename = f"{sf_file.split('.')[0]}-{idx:04d}"

            midi_path = os.path.join(y_path,f'{filename}.midi')
            pm.write(midi_path)

            wav_path = os.path.join(x_path,f'{filename}.wav')
            sf.write(wav_path,x,sr,'PCM_32')

In [ ]:
import requests

discord="https://discord.com/api/webhooks/987570078214144010/MqCbIplVSfwqCcL747t-1Sx6DRuM6_o7QnwmLgw3rpQAO3sB1cY-QUIwu8qCvcJ8KT63"
headers={
    'Content-Type': 'application/json'
}
data = '{"content":"Generating data Completed"}'
    

response = requests.post(
    discord,
    headers=headers,
    data=data.encode('utf8')
)